In [39]:
from viva.cnn.networks.lenet import LeNet
from viva.cnn.networks.xception_transfer import Xception_Transfer
from viva.bottleneck.extract_bottleneck_features import extract_bottleneck_Xception, extract_features_labels
from read_viva import load_viva, DataGen
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import cv2
from pipeline import pretrained_model
from extract_bottleneck_features import extract_Xception

In [2]:
# img_size = (100, 100)
# batch_size = 10
# path = '/media/whatevs/Machine Learning/detectiondata/'
# img_list, box_list = load_viva(path)
# # Splitting into 60/20/20
# train_imgs, test_imgs, train_boxes, test_boxes = train_test_split(
#     img_list, box_list, test_size=0.2)
# train_imgs, valid_imgs, train_boxes, valid_boxes = train_test_split(
#     train_imgs, train_boxes, test_size=0.25)

# # train_gen = 
# bottleneck_model = pretrained_model('Xception', img_size)

# train_generator = generate_batch(train_imgs, train_boxes, img_size=img_size, bottleneck=True, model=bottleneck_model, batch_size=batch_size)
# result = next(train_generator)
# result[0].shape
# print(type(train_generator))
# bottleneck_train_convert = extract_features_labels(bottleneck_model, next(train_generator), batch_size=batch_size)
# valid_generator = generate_batch(
#     valid_imgs, valid_boxes, img_size=img_size, batch_size=batch_size)
# test_generator = generate_batch(
#     test_imgs, test_boxes, img_size=img_size, batch_size=batch_size)

# model_name = 'xception'
# bottleneck_features = extract_bottleneck_Xception(train_generator, valid_generator, test_generator,
#                                                   img_shape=(img_size[0], img_size[1], 3), batch_size=batch_size)
# model = Xception_Transfer.build(bottleneck_features['train']['features'],
#                                 bottleneck_features['train']['labels'])
# bottleneck_features['train']['features']
# bottleneck_train_generator = bottleneck_features
# bottleneck_valid_generator = extract_features_labels(model, valid_generator, batch_size=batch_size)
# bottleneck_test_generator = extract_features_labels(model, test_generator, batch_size=batch_size)



In [19]:
from sliding_window.sliding_window import slide_window, search_windows, heatmap_windows
from read_viva import *

"""
Data Preprocessing
"""
print('[INFO] Preparing the data...')
train_img_list, train_box_list, test_img_list, test_box_list = load_viva('/media/whatevs/Machine Learning/detectiondata/')
# Splitting into train validation 75/25
train_img_list, valid_img_list, train_box_list, valid_box_list = train_test_split(
    train_img_list, train_box_list, test_size=0.25)
print('[INFO] Loaded {} training images'.format(len(train_img_list)))
print('[INFO] Loaded {} validation images'.format(len(valid_img_list)))
print('[INFO] Loaded {} testing images'.format(len(test_img_list)))

[INFO] Preparing the data...
[INFO] Loaded 0 training images
[INFO] Loaded 0 validation images
[INFO] Loaded 0 testing images


In [20]:
# img_size = (128, 128)
# batch_size = 20

# model = LeNet.build(width=128, height=128, depth=3, classes=2,
#                         weights_path='./models/weights.best.sequential_8933.hdf5')
# print(model.summary())

In [21]:
img_size = (128, 128)
batch_size = 20

bottleneck_model = pretrained_model('Xception', img_size)
bottleneck_model._make_predict_function()

train_generator = DataGen(img_size,
                              batch_size,
                              negative=True,
                              bottleneck=True,
                              model=bottleneck_model).generate_train(train_img_list, train_box_list)
valid_generator = DataGen(img_size,
                              batch_size,
                              negative=True,
                              bottleneck=True,
                              model=bottleneck_model).generate_train(valid_img_list, valid_box_list)
test_generator = DataGen(img_size,
                              batch_size,
                              negative=True,
                              bottleneck=True,
                              model=bottleneck_model).generate_train(test_img_list, test_box_list)

x, y, z = bottleneck_model.output.shape[1:]

In [22]:
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=(int(x), int(y), int(z))))
model.add(Dense(2, activation='softmax'))

model.load_weights('./models/weights.best.xception_5933.hdf5')

In [5]:
optimizer = SGD(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

In [42]:
bottle_test = mpimg.imread('./tester3.jpg')
bottle_test = cv2.resize(bottle_test, (128, 128))
print(bottle_test.shape)
bottle_array = img_to_array(bottle_test)
bottle_array = np.expand_dims(bottle_array, axis=0)
preprocess_test = preprocess_input(bottle_array)
print(preprocess_test.shape)

feature_test = extract_Xception(preprocess_test)
print(feature_test.shape)

(128, 128, 3)
(1, 128, 128, 3)
(1, 4, 4, 2048)


In [43]:
model.predict(feature_test)

array([[ 0.49598917,  0.50401092]], dtype=float32)

In [6]:
test_img = mpimg.imread('./tester3.jpg')
print(test_img.shape)
img = test_img[:,:,:3]
print(img.shape)
img = cv2.resize(img, img_size)
print(img.shape)
img = np.expand_dims(img, axis=0)
print(img.shape)
prediction = model.predict(img)
print(prediction)

(167, 127, 3)
(167, 127, 3)
(128, 128, 3)
(1, 128, 128, 3)
[[ 0.  1.]]


In [7]:
prediction = model.predict(img).flatten()
print(prediction)

[ 0.  1.]


In [8]:
prediction = np.argmax(prediction)
print(prediction)

1


In [44]:
def prediction_method(img):
    return np.argmax(model.predict(extract_Xception(img)).flatten()) == 1

In [45]:
from sliding_window.sliding_window import slide_window, search_windows, heatmap_windows, add_heat, apply_threshold, draw_labeled_bboxes
from scipy.ndimage.measurements import label

In [46]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [ ]:
testing_img = mpimg.imread('./tester4.png')
print(testing_img.shape)

xy_window = (300, 300)

windows = slide_window(testing_img, xy_window=xy_window)
print(len(windows))

on_windows = search_windows(prediction_method, 
                            img_size=(128, 128), 
                            img=testing_img.copy(), 
                            windows=windows, 
                            xy_window=xy_window)

boxes_img = draw_boxes(testing_img, on_windows)

(1080, 1920, 3)
Starting sliding window (300, 300) from 0, 0 to 1920, 1080
Finished with 242 windows to slide
242
Starting to search 242 windows


In [ ]:
# heat_img, heat_boxes = heatmap_windows(testing_img, on_windows, 2)

zeros = np.zeros_like(testing_img[:, :, 0]).astype(np.float)
heat = add_heat(zeros, on_windows)
heat_thresh = apply_threshold(heat, 2)
heatmap = np.clip(heat_thresh, 0, 255)
labels = label(heatmap)

heat_img, heat_boxes = draw_labeled_bboxes(testing_img.copy(), labels)

plt.figure(figsize=(16,8))
plt.subplot(1, 4, 1)
plt.title('classified')
plt.imshow(boxes_img)
plt.subplot(1, 4, 2)
plt.title('heatmap')
plt.imshow(heat)
plt.subplot(1, 4, 3)
plt.title('heatmap threshold')
plt.imshow(heat_thresh)
plt.subplot(1, 4, 4)
plt.title('final')
plt.imshow(heat_img)
plt.show()